## Imports

In [1]:
import os
import pandas as pd
import numpy as np

## List Files to Clean

In [2]:
# Import current list of files and their cleaning status
df_combined_data = pd.read_csv('./Data/combined_files.csv')

In [3]:
# Import names of all "Coronavirus Tweets" files currently in /Data directory
# .find method taken from:
# https://www.afternerd.com/blog/python-string-contains/
data_files = list(filter(lambda file: (file.find('Coronavirus Tweets') != -1), os.listdir('./Data')))

# Filter list of files to new files only
new_files = list(filter(lambda file: file not in df_combined_data['file'].values , data_files))

In [4]:
# Create dataframe of new files. Set added status as 0
df_new_files = pd.DataFrame([new_files,[0]*len(new_files)], index = ['file','added']).T

# Append new files dataframe to existing dataframe
df_combined_data = df_combined_data.append(df_new_files).reset_index(drop = True)

## Build Cleaning Steps

In [7]:
# Create dictionary of US state names and codes from:
# https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'American Samoa': 'AS', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA',
    'Guam': 'GU', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI',
    'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Northern Mariana Islands':'MP', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR',
    'Pennsylvania': 'PA', 'Puerto Rico': 'PR', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virgin Islands': 'VI', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

In [8]:
# Define function to filter dataframe to tweets that are:
# 1) from the US, 2) with valid locations 3) more granular than country, and 4) in english
def get_US_data(df):
    mask_us = (df['country_code'] == 'US')
    mask_place = (df['place_full_name'].notna())
    mask_country = (df['place_type'] != 'country')
    mask_eng = (df['lang'] == 'en')
    return df.loc[mask_us & mask_place & mask_country & mask_eng,:]

In [9]:
# Define function to correct data types and drop unnecessary columns
def correct_columns_kag(df):
    # Set datetime columns as pandas datetime type
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['account_created_at'] = pd.to_datetime(df['account_created_at'])
    
    # Create boolean column for whether a tweet is a reply and drop all other reply columns
    df['is_reply'] = df['reply_to_user_id'].notna()
    
    # Change spelling of favorites_count to match IEEE
    df.rename(columns = {'favourites_count': 'favorite_count'}, inplace = True)
                         
    # Remove all unneeded columns
    df = df.drop(columns = ['account_lang','reply_to_user_id','reply_to_status_id','reply_to_screen_name','source',
                            'user_id','is_quote'])
    
    return df

In [10]:
# Define function to correct names of columns from IEEE dataset
def correct_names_ieee(df):
    # Rename columns to match original dataset
    df.rename(columns = {'user_followers_count' : 'followers_count',
                         'user_friends_count' : 'friends_count',
                         'place' : 'place_full_name',
                         'user_screen_name' : 'screen_name',
                         'id' : 'status_id',
                         'user_verified' : 'verified',
                         'user_created_at': 'account_created_at'
    }, inplace = True)
    return df

In [11]:
# Define function to correct data types and drop unnecessary columns
def correct_columns_ieee(df):
    # Set datetime columns as pandas datetime type
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['account_created_at'] = pd.to_datetime(df['account_created_at'])
    
    # Create boolean column for whether a tweet is a reply and drop all other reply columns
    df['is_reply'] = df['in_reply_to_user_id'].notna()
    df = df.drop(columns = ['in_reply_to_user_id','in_reply_to_status_id','in_reply_to_screen_name'])
    
    # Recreate the boolean is_retweet column
    df['is_retweet'] = df['retweet_id'].notna()
    
    # Recreate country_code and place_type column
    df['country_code'] = df['state'].map(lambda state: 'US' if state != 'unspecified' else 'unspecified')
    df['place_type'] = 'unspecified'
    
    # Remove all unneeded columns
    df = df.drop(columns = ['coordinates','media','possibly_sensitive','hashtags','urls','user_default_profile_image',
                            'user_description','source','retweet_screen_name','user_favourites_count','user_listed_count'
                            ,'user_location','user_name','user_screen_name.1','user_statuses_count','user_time_zone'
                            ,'user_urls','retweet_id','tweet_url'])
    return df

In [12]:
# Define function to assign city names where identifiable
def split_city(row, state_dict):
    # Check whether the place name is split consistently with city/state names
    if len(row['place_full_name'].split(', ')) != 2:
        return 'unspecified'
    else:
        # Separate place name into components
        first_split = row['place_full_name'].split(', ')[0]
        second_split = row['place_full_name'].split(', ')[1]
        
        # Create masks for whether the first or second split are identifiable as state names or state codes
        mask_second = (second_split not in state_dict.values() and second_split not in state_dict.keys())
        mask_first = (first_split not in state_dict.values() and first_split not in state_dict.keys())
        
        # Assign city names based on place types and whether splits are identified as state names or codes
        if row.get('place_type') == 'city':
            return first_split
        elif row.get('place_type') == 'neighborhood' and mask_second:
            return second_split
        elif row.get('place_type') == None and mask_first:
            return first_split
        else:
            return 'unspecified'

In [13]:
# Define function to assign state codes where identifiable
def split_state(row, state_dict):
    # Set state as unspecified if it does not follow the standard location format
    if len(row['place_full_name'].split(', ')) != 2:
        return 'unspecified'
    else:
        # Separate place name into components
        first_split = row['place_full_name'].split(', ')[0]
        second_split = row['place_full_name'].split(', ')[1]
        
        # Create separate lists of state names and codes
        state_codes = state_dict.values()
        state_names = state_dict.keys()
        
        # Attempt to identify state codes and state names within first and second splits based on place type
        if row.get('place_type') == None and (second_split in state_codes):
            return second_split
        elif row.get('place_type') == None and (second_split in state_names):
            return state_dict[second_split]
        elif row.get('place_type') == 'city' and second_split in state_codes:
            return second_split
        elif row.get('place_type') == 'neighborhood' and second_split in state_codes:
            return second_split
        elif row.get('place_type') == 'neighborhood' and second_split in state_names:
            return state_dict[second_split]
        elif (row.get('place_type') == 'admin' or row.get('place_type') == None) and first_split in state_names:
            return state_dict[first_split]
        else:
            return 'unspecified'

## Loop Through Files for Cleaning

In [19]:
# Read in the current combined US data CSV. If none exists, instantiate a dataframe for future joins
try:
    df_us = pd.read_csv('./data/combined_us.csv', index_col = 0)
except:
    df_us = pd.DataFrame()
    
# Loop through all files that have yet to be added to the combined 
for file in df_combined_data.loc[df_combined_data['added'] == 0, 'file']:
    # Read file from CSV
    df_file = pd.read_csv(f'./data/{file}')
    
    # Filter data down to only English tweets from the US with valid locations
    # For IEEE formatted data, filter down only to valid place names
    if file.find('IEEE') != -1:
        df_file = correct_names_ieee(df_file)
        df_file = df_file[df_file['place_full_name'].notna()]
    else:
        df_file = get_US_data(df_file)
    
    # Create city and state columns
    df_file['city'] = df_file.apply(lambda row: split_city(row,us_state_abbrev), axis = 1)
    df_file['state'] = df_file.apply(lambda row: split_state(row,us_state_abbrev), axis = 1)
    
    # Correct data types and missing columns    
    if file.find('IEEE') != -1:
        df_file = correct_columns_ieee(df_file)
        df_file = get_US_data(df_file)
    else:
        df_file = correct_columns_kag(df_file)
    
    # Drop remaining nulls
    # Commenting this step out for now to clarify what needs to be dropped
#     df_file = df_file.dropna()
    
    # Add the data to the existing combined US data
    df_us = df_file.append(df_us)
    
    # Update the combined data reference table to indicate the file has been read
    df_combined_data.loc[df_combined_data['file'] == file, 'added'] = 1

In [24]:
# Drop duplicate records
df_us = df_us.drop_duplicates(subset = 'status_id')

## Update Files

In [25]:
# Update combined US data file
df_us.to_csv('./Data/combined_us.csv', index = False)

In [21]:
# Update reference file for included data
df_combined_data.to_csv('./Data/combined_files.csv', index = False)